In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle

from IPython.display import display_html
from keras.models import load_model, Model
from matplotlib.colors import rgb2hex
import matplotlib.pyplot as plt
import numpy as np

from masterthesis.features.build_features import file_to_sequence
from masterthesis.utils import ATTENTION_LAYER, DATA_DIR, MODEL_DIR, RESULTS_DIR, load_split

Using TensorFlow backend.


In [3]:
def show_weighted(tokens, weights):
    cmap = plt.cm.coolwarm
    ents = ['<p style="max-width: 80ch">']
    for token, weight in zip(tokens, weights):
        col = rgb2hex(cmap(weight))
        ent = '<span style="color: %s">%s</span>' % (col, token)
        ents.append(ent)
    ents.append('</p>')
    return ' '.join(ents)

In [4]:
def tokens_in_file(filepath):
    with filepath.open() as f:
        for line in f:
            for token in line.strip().split():
                yield token

In [5]:
def file_with_attention(filepath, attention_model, w2i, maxlen=None):
    x = file_to_sequence(700, filepath, w2i)[np.newaxis, :]
    weights = attention_model.predict(x)[0]
    weights -= weights.min()
    weights /= weights.max()
    if maxlen is not None:
        weights = weights[:maxlen]
    unk_idx = w2i['__UNK__']
    tokens = (token if idx != unk_idx else 'UNK' for token, idx in zip(tokens_in_file(filepath), x[0]))
    return show_weighted(tokens, weights)

In [6]:
model_name = 'rnn-nli-26542571_9'

# model_name = 'rnn_nli-25717032'

In [7]:
w2i = pickle.load((MODEL_DIR / (model_name + '_model_w2i.pkl')).open('rb'))
model = load_model(str(MODEL_DIR / (model_name + '_model.h5')))

In [8]:
attention_model = Model(inputs=model.input, outputs=model.get_layer(ATTENTION_LAYER).output)

In [15]:
dev = load_split('dev')

results = pickle.load((RESULTS_DIR / (model_name + '.pkl')).open('rb'))
correct_predictions = dev[results.true == results.predictions]
print('Accuracy: %.3f' % (len(correct_predictions) / len(dev)))
langs = ['russisk', 'vietnamesisk', 'engelsk', 'somali']
sample_indices = [correct_predictions[correct_predictions.lang == l].index[0]
                  for l in langs]
samples = correct_predictions.loc[sample_indices]
display(samples)
filenames = [DATA_DIR / 'txt' / (fn + '.txt') for fn in samples.filename]

Accuracy: 0.463


,age,cefr,filename,gender,lang,num_tokens,split,testlevel,title,topic
182,32,B1/B2,h0185,kvinne,russisk,536,dev,Høyere nivå,Helse og livskvalitet,helse
1457,24,B1/B2,s0628,kvinne,vietnamesisk,420,dev,Språkprøven,En hyggelig opplevelse,opplevelse
167,39,B2,h0169,kvinne,engelsk,595,dev,Høyere nivå,Helse og livskvalitet,helse
1375,26,A2,s0546,mann,somali,176,dev,Språkprøven,Flytting,familie flytting


In [16]:
for filename in filenames:
    display_html('<h2>%s</h2>' % filename.stem, raw=True)
    html = file_with_attention(filename, attention_model, w2i, maxlen=300)
    with open(filename.stem + '.html', 'w') as f:
        print("""
<!DOCTYPE html>
<html lang="nb">
<head>
  <meta charset="utf-8">
  <title>%s</title>
  <style>
  body {
    font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
    font-size: 14px;
    line-height: 20px;
    text-align: left;
  }
  </style>
</head>
<body>
""" % filename.stem, file=f)
        print(html, file=f)
        print('</body></html>', file=f)
    display_html(html, raw=True)

h0185

Oppgave A | I dag bor vi i en fantastisk tid . Man får masse muligheter . Vi har mye av det som besteforeldrene våre kunne bare drømme om . Det er helt utrolig å tenke på hva vi har oppnådd . Men samtidig har vi fått mange problemer . En av de problemene er vår egen helse . I dag har vi et godt utviklet UNK og mange forskjellige UNK . Men alt dette hjelper neppe så mye hvis hvert enkeltmenneske ikke tenker på sin egen helse . Vi må huske at livet er en gave og en god helse hjeper veldig mye for å nytte livet . Derfor er det viktig å tenke på den måten vi lever på . Først og fremst bør man tenke på hva man spiser . Undersøkelser viser at folk har færre UNK i de landene hvor man spiser mye grønnsaker og frukt . Å ha UNK UNK til middag UNK for brus er også veldig sunt . Ikke minst viktig er det å ha litt mosjon og trim . I dag bruker man mye transport . Vi kjører veldig mye selv om vi ikke trenger det . Jeg var veldig overasket da jeg kom første gang til Norge og oppdaget at mennesker bruker bil for å komme Mange har sånn jobb som at de trenger å sitte hele arbeidsdag . Da må man begynne å bli bekymret for sin kropp . Det er best å gå på tur eller sykle . Noen trener på sportklub eller svømmer i UNK . Godt UNK er også en av de viktigste momentene . Vi forlenger livet når vi smiler eller tenker positivt . Det virker veldig UNK å ha en god helse og leve lengre . Livet er spent . Man kommer i verden ,

s0628

UNK UNK . | Jeg skal fortelle deg om en hyggelig opplevelse jeg har hatt – et lykkelig minne . Jeg tror at alle mennesker har hatt sine hyggelige opplevelser . Men for meg er det ikke så mange . Jeg vokste opp i en liten UNK i en liten landsby i Vietnam . Familien min var far , mor , jeg og de andre fire søsken . Vi var fattige mennesker . Foreldrene mine måtte jobbe hele tiden i vår gård for å gi oss mat , klær , osv. . Selv om vi hadde et hardt live t , ville foreldrene mine at barna kunne gå på skolen . Derfor måtte jeg og fire søsken ga både gå på skolen og jobbe . Men i Vietnam må man UNK selv hvis man går på skolen : Det er alt : UNK , bøker , UNK . Det var alt for mye for oss . Vi hadde liten inntekt for masse ting . Så måtte to av de eldste søsken slutte skolen . Det var bare jeg og de andre som gikk videre . Da jeg var bare ti år gammel men jeg forstod at foreldrene mine ønsket at barna sine kunne gå på universitet og det førte til at de kunne finne bedre bra jobb og bedre live etterpå . Jeg prøvde å studere , prøvde å gjøre være flink på skolen for å kunne se foreldrene skulle være glad . Jeg prøvde , prøvde ... Da jeg fikk et brev fra et universitet i hjemmelandet mitt , UNK mitt hjerte så fort . Dette brevet viste at jeg har bestått UNK på universitetet ( I Vietnam må du ta en veldig vanskelig prøve for å søke universitet ) . Det var så glad jeg kan være .

h0169

Oppgave A | Din helsetilstand har mye å si om hvordan du opplever dit livskvalitet . Det er mange moter å ta være på sin egen helse og UNK viser at mange mennesker lever lenger enn tidligere på grunn av ny kunnskap og utvikling innenfor medisin og teknologi . Spørsmålet er : hvordan kan vi ta være på vår helse og er det greit å oppnå en høy alder ? Det er flere moter at du kan ta være på din egen helse . Først kan du passe på det du spiser . Alle vet nå at fett er usunt , men hvor mange leser UNK for å vite om UNK av maten de spiser ? Å bli kjent med kunstige UNK er også viktig . Noen av disse midler er ikke farlig men , på den andre siden , er andre UNK . Annen kan du lærer om de tingene du trenger for å bli sterk og sunt i kroppen . Mye er skrevet i det siste om , f.eks. vitaminer og god og skadelig kolesterol . Les ! Hvis du tar være på kroppen din , skal du kanskje oppnå en høy alder . Men , er dette nødvendigvis en god ting ? Jeg synes det har mye å si for samfunnet . For det første , å ha eldre mennesker som en del av samfunnet er bra for samfunnet når det gjelder livserfaring . Vi har , eller kan , lære mye fra de eldre . På den andre siden , er det en stor belastning for samfunnet ( på grunn av bekostningen til helsevesen ) å ha så mange eldre mennesker . Ofte trenger eldre mennesker mye omsorg senere i livet og mange opplever langvarig opphold i sykehus eller i UNK For de eldre selv er

s0546

En tekst om å flytte . | Jeg har flyttet mange ganger da jeg bodde i @sted . Jeg har også UNK par ganger på grunn av krig som er en del av flytting . Jeg har flytet fra @sted til Norge UNK jeg har ikke flytet , jeg har UNK til Norge men ja å UNK er nesten same som å flytte . I UNK å flytte fra et sted til annet sted er ikke lett det koster mye tid , kraft og kanskje mye penger . Det UNK om å flytte er at det jobben ikke blir bare dagen som du flytter men - fortsetter dager , uker eller måneder etter at du flytet . Du må sette alt på plass eller ordne mange ting som må UNK . Og når du kommer et land eller et sted som bor folk som har en UNK kultur enn du har er det mest vanskeligst av UNK . Det forstår bare de som har opplevd . Jeg har selv opplevd men nå går det bra .